<a href="https://colab.research.google.com/github/BosenkoTM/OOAP-2020/blob/master/%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%80%D0%B0%D1%81%D1%85%D0%BE%D0%B4%D0%BE%D0%B2_%D0%BD%D0%B0_%D1%86%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8E_%D1%88%D0%BA%D0%BE%D0%BB_API_16_01_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ТЕМА: Сравнительный анализ расходов на цифровизацию школ 

**Краткое описание проблематики:**

**Цифровая образовательная среда** - это совокупность специально организованных педагогических условий развития личности, при которой инфраструктурный, содержательно-методический и коммуникационно-организационный компоненты функционируют на основе цифровых технологий. Вместе с тем, в Российской Федерации наблюдается отставание в цифровом обновлении образования, что позволит нам догнать передовые страны по качеству образования. Разработанный набор индикаторов, характеризующий цифровизацию образовательной среды, и дальнейшая визуализация помогут наглядно показать состояние и готовность образовательной среды с новым реалиям. 


Ссылка на проект "Госрасходы": https://spending.gov.ru/

Ссылка на документацию API: https://code.ach.gov.ru/sgr/spending-api-docs/-/wikis/home

##Инструменты##

Python 3 (https://www.python.org/downloads/)

Jupyter Notebook (https://jupyter.org/)

Google Colaboratoty (https://colab.research.google.com/notebooks/welcome.ipynb?hl=ru)

JSON Viewer (приложение для Google Chrome)
(https://chrome.google.com/webstore/detail/json-viewer/gbmdgpbipfallnflgajpaliibnhdgobh?hl=ru)

## Что такое API? ##

**UI - User Interface**, интерфейс пользователя, пользовательский интерфейс — интерфейс, обеспечивающий передачу информации между пользователем-человеком и программно-аппаратными компонентами компьютерной системы.


**API - Application Programming Interface**, программный интерфейс приложения.


**Открытые API** - это стандартизированный набор программных инструментов, которые обеспечивают взаимодействие между приложениями. С помощью открытых API можно получить доступ к функционалу и информации владельца интерфейса.


**API** - это то, что может делать приложение по просьбе других приложений.




## Написание кода для запросов к API

**Requests** - python-библиотека для отправки всех видов HTTP-запросов.

Документация: https://requests.readthedocs.io/en/master/

**Pandas** - python-библиотека для обработки и анализа данных.

Документация: https://pandas.pydata.org/

In [1]:
import requests 
import pandas as pd  

In [ ]:
#импортируем библиотеки, которые нам пригодятся
# установка библиотек
#!pip install requests
#!pip install pandas

import requests             #чтобы сделать запросы к API
import pandas as pd         #чтобы обработать выдачу, сформировать и выгрузить таблицу

**Ожидаемый результат**:

Интерактивная карта субъектов Российской Федерации по стоимости закупаемого оборудования в рамках внедрения цифровой образовательной среды, а также построение рейтинга поставщиков 


# 1. Стоимость закупаемого оборудования

На сайте https://spending.gov.ru/np/E/fp/E4/contracts/ находим все контракты в рамках нацпроекта Образование, подкаталог Цифровая образовательная среда. Определяем шифр контрактов по данному направлению - **E4** (инфа по шифрам к  запросам через  API  для формирования JSON  тут https://code.ach.gov.ru/sgr/spending-api-docs/-/wikis/%D0%9D%D0%B0%D1%86%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D1%8B)

In [ ]:
#делаем запрос, результат (ответ API) сохраняем в переменную response в 

response = requests.get('https://api.spending.gov.ru/v1/natprojects/contracts?fp_id=E4')

#результат (ответ API) лучше проверить, особенно при больших выгрузках

print(response)


<Response [200]>


In [ ]:
#сохраняем результат в переменную contracts с помощью метода json

contracts = response.json()

print(contracts['total']) #  всего контрактов

348


Посмотрим результат поиска всех контрактов  в рамках нацпроекта Образование, подкаталог Цифровая образовательная среда

In [ ]:
contracts

##3. Постраничная навигация##

**total** - найдено записей всего;

**page** - страница в выдаче;

**perpage** - количество записей в одном запросе (max - 50).\

Для перемещения по страницам добавляем к запросу параметр **page**.

https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&daterange=01.01.2019-01.01.2020&page=2 (3, 4, 5, 6...)

Сколько страниц нужно пройти, чтобы выгрузить все контракты? 
**total // 50 + 1**

In [ ]:
#получаем значение total из json-файла contracts

total = contracts['total']
total

348

In [ ]:
#получаем количество страниц в выдаче 
pages = total // 50 + 1 #два бэкслеша - деление без остатка
pages

7

In [ ]:
# теперь можно создать цикл, который будет подставлять номер страницы в запрос и пройдет по всем страницам
# для этого используем функцию range и форматирование строки

for page in range(1, 8):
  response = requests.get(f'https://api.spending.gov.ru/v1/natprojects/contracts?fp_id=E4&page={page}')
  print(page)
  print(response)

1
<Response [200]>
2
<Response [200]>
3
<Response [200]>
4
<Response [200]>
5
<Response [200]>
6
<Response [200]>
7
<Response [200]>


##4. Парсинг JSON файла

**JSON (JavaScript Object Notation)** – текстовый формат для представления значений и объектов. Первоначально создан для JavaScript, но многие другие языки также имеют библиотеки, которые могут работать с ним. 

**Хранит структурированную информацию (ключ: значение). Доступен для чтения как программой, так и человеком.**

**По сути, JSON-файл - это вложенные в друг друга словари и списки. Сдлявари в python обозначаются фигурнами скобками {}, спики - квадратными скобками [ ]**.

Для написания корректного кода, необходимо пристально изучить структуру JSON-файла.


In [ ]:
for key in contracts.keys():
    print(key)

count
total
page
items


In [ ]:
#одна страница выдачи - это список из 50 контрактов

contracts['items']

[{'budget_code': '26020001',
  'budget_level_code': '2',
  'budget_name': 'Бюджет Белгородской области',
  'contract_stage': 'EC',
  'contract_subject': None,
  'currency': 'RUB',
  'customer_inn': '3123316994',
  'customer_kpp': '312301001',
  'customer_name': 'ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ',
  'customer_spz': '01262000034',
  'fp_ids': ['E4'],
  'fp_info': [{'fp_id': 'E4',
    'fp_price': 2895911.83,
    'grbs_code': '810',
    'grbs_name': None,
    'kbk': '8100702022E4E0000242'}],
  'full_price': 2895911.83,
  'load_id': 236416,
  'max_id': 484506,
  'np_ids': ['E'],
  'np_info': [{'np_id': 'E', 'np_price': 2895911.83}],
  'products': [{'name': 'Принтер',
    'okei_code': '796',
    'okei_name': 'Штука',
    'okpd2_code': None,
    'okpd2_name': None,
    'price': 14850.82,
    'quantity': 194.0},
   {'name': 'Принтер',
    'okei_code': '796',
    'okei_name': 'Штука',
    'okpd2_code': None,
    'okpd2_name': None,
    'price': 14852.75,
    'quantity': 1.0}],
  're

In [ ]:
#посмотрим на длину этого списка

len(contracts['items'])

In [ ]:
#получение необходимых данных 
contracts['items'][0]['fp_info']
#contracts['contracts']['data'][0]['regNum'] #регистрационный номер контракта
# contracts['contracts']['data'][0]['signDate'] #дата подписания контракта
# contracts['contracts']['data'][0]['customer'] #данные заказчика
# contracts['contracts']['data'][0]['customer']['fullName'] #наименование заказчик
# contracts['contracts']['data'][0]['customer']['inn'] #ИНН заказчика
# contracts['contracts']['data'][0]['customer']['kpp'] #КПП заказчика
# contracts['contracts']['data'][0]['customer']['postalAddress'] #адрес заказчика

[{'fp_id': 'E4',
  'fp_price': 2895911.83,
  'grbs_code': '810',
  'grbs_name': None,
  'kbk': '8100702022E4E0000242'}]

In [ ]:
listtt=contracts['items'][0]['fp_info']
a={}
a=listtt[0]
a

{'fp_id': 'E4',
 'fp_price': 2895911.83,
 'grbs_code': '810',
 'grbs_name': None,
 'kbk': '8100702022E4E0000242'}

In [ ]:
contracts['items'][0]['suppliers']

[{'supplier_inn': '3123314130',
  'supplier_kpp': '312301001',
  'supplier_name': 'ООО "РЕСУРС-МЕДИА"'}]

In [ ]:
contracts['items'][0]['customer_name']


'ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ'

## Выгрузка данных ## 

Объединяем все предыдущие шаги в одном коде. Пример выгрузки "по продуктам".

In [ ]:
contracts_tab = [] #в этот список будем складывать информацию о каждом контракте
for page in range(1, 8):
  response = requests.get(f'https://api.spending.gov.ru/v1/natprojects/contracts?fp_id=E4&page={page}')
  print(page)
  print(response)
  contracts = response.json()
  lst = contracts['items']
  for cnt in lst:
    for prod in cnt['fp_info']:
      list1=cnt['fp_info']
      a={}
      a=list1[0]
      one_cont = []
      one_cont.append(cnt['customer_name'])
      #one_cont.append(cnt['customer_inn'])
      #one_cont.append(cnt['customer_kpp'])
      one_cont.append(a['fp_price'])
      list2=cnt['suppliers']
      b={}
      b=list2[0]
      one_cont.append(b['supplier_name'])
      one_cont.append(cnt['year'])
      contracts_tab.append(one_cont)


1
<Response [200]>
2
<Response [200]>
3
<Response [200]>
4
<Response [200]>
5
<Response [200]>
6
<Response [200]>
7
<Response [200]>


In [ ]:
# проверим выгруженные значения в итоговом списке

contracts_tab


[['ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ',
  2895911.83,
  'ООО "РЕСУРС-МЕДИА"',
  '2019'],
 ['ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ',
  5261100.0,
  'ООО "ИК ФОРСИС"',
  '2019'],
 ['ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ',
  5502535.16,
  'ООО "Консалтинговая группа Овечкин и партнеры"',
  '2019'],
 ['ГОСУДАРСТВЕННОЕ КАЗЁННОЕ УЧРЕЖДЕНИЕ ЯМАЛО-НЕНЕЦКОГО АВТОНОМНОГО ОКРУГА "РЕГИОНАЛЬНЫЙ ЦЕНТР ОЦЕНКИ КАЧЕСТВА ОБРАЗОВАНИЯ"',
  2800000.0,
  'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ИНСТИТУТ РАЗВИТИЯ ПЕРСОНИФИЦИРОВАННЫХ СИСТЕМ УПРАВЛЕНИЯ"',
  '2019'],
 ['ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ',
  591317.0,
  'Индивидуальный предприниматель Мезенина Аксана Васильевна',
  '2019'],
 ['ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ',
  10866245.5,
  'ООО "ЛЛС ГРУПП"',
  '2019'],
 ['ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ',
  23143961.3,
  'ООО "ЛЛС ГРУПП"',
  '2019'],
 ['ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ',
  1403664.0,
  'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВ

In [ ]:
# с помощью метода pd.DataFrame преобразуем список в датафрейм (таблицу)

frame = pd.DataFrame(contracts_tab)

In [ ]:
frame

,0,1,2,3
0,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,2.895912e+06,"ООО ""РЕСУРС-МЕДИА""",2019
1,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,5.261100e+06,"ООО ""ИК ФОРСИС""",2019
2,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,5.502535e+06,"ООО ""Консалтинговая группа Овечкин и партнеры""",2019
3,ГОСУДАРСТВЕННОЕ КАЗЁННОЕ УЧРЕЖДЕНИЕ ЯМАЛО-НЕНЕ...,2.800000e+06,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНСТ...",2019
4,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,5.913170e+05,Индивидуальный предприниматель Мезенина Аксана...,2019
...,...,...,...,...
349,МИНИСТЕРСТВО ОБРАЗОВАНИЯ МОСКОВСКОЙ ОБЛАСТИ,1.420000e+08,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИБС ...",2019
350,МИНИСТЕРСТВО ПРОСВЕЩЕНИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ,7.000000e+07,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""МИХАЙЛОВ И ПАРТНЕРЫ. УПР...",2020
351,МИНИСТЕРСТВО ОБРАЗОВАНИЯ САХАЛИНСКОЙ ОБЛАСТИ,3.480322e+07,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО «РОСТЕЛЕКОМ» (С...,2021
352,МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ АСТРАХАНСКОЙ ...,2.359869e+07,Публичное акционерное общество «Мегафон»,2020


In [ ]:
#зададим названия столбцам

frame.rename(columns={0: 'Заказчик', 1: 'Стоимость_оборудования', 2: 'Поставщик', 3: 'Год'}, inplace=True)

In [ ]:
#посмотрим, что получилось

frame.head(50)

,Заказчик,Стоимость_оборудования,Поставщик,Год
0,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,2.895912e+06,"ООО ""РЕСУРС-МЕДИА""",2019
1,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,5.261100e+06,"ООО ""ИК ФОРСИС""",2019
2,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,5.502535e+06,"ООО ""Консалтинговая группа Овечкин и партнеры""",2019
3,ГОСУДАРСТВЕННОЕ КАЗЁННОЕ УЧРЕЖДЕНИЕ ЯМАЛО-НЕНЕ...,2.800000e+06,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНСТ...",2019
4,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,5.913170e+05,Индивидуальный предприниматель Мезенина Аксана...,2019
5,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,1.086625e+07,"ООО ""ЛЛС ГРУПП""",2019
6,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,2.314396e+07,"ООО ""ЛЛС ГРУПП""",2019
7,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,1.403664e+06,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЛЕОС...",2019
8,ДЕПАРТАМЕНТ ОБРАЗОВАНИЯ БЕЛГОРОДСКОЙ ОБЛАСТИ,4.427216e+06,"ООО ""ЛЛС ГРУПП""",2019
9,ГОСУДАРСТВЕННОЕ КАЗЁННОЕ УЧРЕЖДЕНИЕ ЯМАЛО-НЕНЕ...,3.960000e+06,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИРСОФТ""",2019


In [ ]:
#сохраним выгрузку в таблицу формата csv

frame.to_csv('contracts.csv', index=False)

In [ ]:
from google.colab import files
with pd.ExcelWriter('contracts.xlsx') as writer:  
    frame.to_excel(writer, sheet_name='contracts')
files.download('contracts.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2 Рейтинг компаний